In [40]:
path = "abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/"


customer = spark.read.option("header", "true").csv(path + "customer")
product  = spark.read.option("header", "true").csv(path + "product")
department  = spark.read.option("header", "true").csv(path + "department")
category  = spark.read.option("header", "true").csv(path + "category")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 42, Finished, Available)

In [41]:
from pyspark.sql.functions import col,row_number,lit,current_timestamp, to_timestamp
from pyspark.sql.window import Window
import pyspark.sql.functions as F

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 43, Finished, Available)

# Manage dim_customer

In [42]:
dim_customer = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_customer")


StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 44, Finished, Available)

In [43]:
#register as teamview
customer.createOrReplaceTempView("customer")

new_customer = spark.sql("""
    SELECT * 
    FROM customer c
    WHERE c.customer_id NOT IN (
        SELECT customer_id FROM dim_customer
    )
""")
changed_customer  = spark.sql("""
SELECT c.customer_id, c.customer_segment, c.customer_fullname
FROM customer c JOIN dim_customer  dc
ON c.customer_id = dc.customer_id 
WHERE (c.customer_fullname != dc.customer_fullname OR c.customer_segment != dc.customer_segment) AND dc.is_valid = 'true'

""")


StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 45, Finished, Available)

In [44]:
# get max value of customer key
max_customer_key_df = spark.sql("""
    SELECT MAX(customer_key) AS max_customer_key
    FROM dim_customer
""")


max_customer_key = max_customer_key_df.collect()[0]["max_customer_key"]

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 46, Finished, Available)

In [45]:
# combine file 
all_customer = new_customer.union(changed_customer )

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 47, Finished, Available)

In [46]:



# Set start_date
all_customer = all_customer.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("customer_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
all_customer = all_customer.withColumn("customer_key",max_customer_key +  row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("customer_id").orderBy("start_date")

# Set end_date
all_customer = all_customer.withColumn("end_date", to_timestamp(F.lit("9999-12-31")) )


all_customer = all_customer.withColumn("is_valid", lit(True))

all_customer = all_customer.select("customer_key", "customer_id", "customer_segment","customer_fullname", "start_date","end_date","is_valid")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 48, Finished, Available)

In [47]:
update_is_valid = spark.sql("""
SELECT dc.customer_key, dc.customer_id, dc.customer_segment, dc.customer_fullname,dc.start_date,dc.end_date , dc.is_valid
FROM customer c JOIN dim_customer  dc
ON c.customer_id = dc.customer_id 
WHERE (c.customer_fullname != dc.customer_fullname OR c.customer_segment != dc.customer_segment) AND dc.is_valid = 'true'

""")




StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 49, Finished, Available)

In [48]:
# except it from old df
dim_customer = dim_customer.exceptAll(update_is_valid)

update_is_valid = update_is_valid.withColumn("is_valid", lit(False))
update_is_valid = update_is_valid.withColumn("end_date", current_timestamp())

update_is_valid.createOrReplaceTempView("update_is_valid")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 50, Finished, Available)

In [49]:

dim_customer = dim_customer.union(update_is_valid)
dim_customer = dim_customer.union(all_customer)


display(dim_customer)

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 51, Finished, Available)

SynapseWidget(Synapse.DataFrame, c341998c-f963-4205-b961-e73d073ff3fe)

# manage product

In [50]:
dim_product = spark.sql("SELECT * FROM dim_product")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 52, Finished, Available)

In [51]:
max_product_key_df = spark.sql("""
SELECT MAX(product_key) as max_product_key FROM
dim_product
""")

max_product_key = max_product_key_df.collect()[0]["max_product_key"]





StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 53, Finished, Available)

In [52]:
display(product)

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 54, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9f3e68fd-83d9-436f-a571-ae4a056a0cad)

In [53]:
#register as teamview
product.createOrReplaceTempView("product")

new_product = spark.sql("""
    SELECT * 
    FROM product p
    WHERE p.product_card_id NOT IN (
        SELECT product_card_id FROM dim_product
    )
""")
changed_product  = spark.sql("""
SELECT p.product_card_id, p.product_name, p.product_price, p.product_status, p.category_id
FROM product p JOIN dim_product dp
ON p.product_card_id = dp.product_card_id
WHERE (p.product_name != dp.product_name OR p.product_price != dp.product_price OR p.product_status != dp.product_status OR p.category_id != dp.category_id) AND dp.is_valid = 'true'

""")


StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 55, Finished, Available)

In [54]:
# combine file 
all_product = new_product.union(changed_product)

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 56, Finished, Available)

In [55]:



# Set start_date
all_product = all_product.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("product_card_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
all_product = all_product.withColumn("product_key",max_product_key +  row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("product_card_id").orderBy("start_date")

# Set end_date
all_product = all_product.withColumn("end_date", to_timestamp(F.lit("9999-12-31")) )


all_product = all_product.withColumn("is_valid", lit(True))

all_product = all_product.select("product_key", "product_card_id", "product_name", "product_price","product_status","category_id","start_date", "end_date", "is_valid")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 57, Finished, Available)

In [56]:
display(all_product)

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 58, Finished, Available)

SynapseWidget(Synapse.DataFrame, 64fc8a97-1bbd-44ec-88d2-3865ab09a25c)

In [57]:
update_is_valid_product = spark.sql("""
SELECT dp.product_key, dp.product_card_id, dp.product_name, dp.product_price, dp.product_status, dp.category_id, dp.start_date, dp.end_date, dp.is_valid
FROM  dim_product  dp JOIN product p
ON dp.product_card_id = p.product_card_id
WHERE (p.product_name != dp.product_name OR p.product_price != dp.product_price OR p.product_status != dp.product_status OR p.category_id != dp.category_id) AND dp.is_valid = 'true'                                   
                                    
                                    
""")


StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 59, Finished, Available)

In [58]:
dim_product = dim_product.exceptAll(update_is_valid_product)
update_is_valid_product = update_is_valid_product.withColumn("is_valid", lit(False))
update_is_valid_product = update_is_valid_product.withColumn("end_date", current_timestamp())
update_is_valid_product.createOrReplaceTempView("update_is_valid_product")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 60, Finished, Available)

In [59]:


dim_product = dim_product.union(update_is_valid_product)
dim_product = dim_product.union(all_product)

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 61, Finished, Available)

# manage department

In [60]:
department_data =  spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.department")

department_data.createOrReplaceTempView("department_data")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 62, Finished, Available)

In [61]:
# register teamview
department.createOrReplaceTempView("department")
new_department = spark.sql("""
SELECT * FROM department 
WHERE department_id NOT IN (
    SELECT department_id FROM department_data
)
""")

changed_department = spark.sql("""
SELECT d.department_id, d.department_name FROM 
department d JOIN department_data dd ON
d.department_id = dd.department_id 
WHERE d.department_name != dd.department_name
""")

changed_department.createOrReplaceTempView("changed_department")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 63, Finished, Available)

In [62]:
# find max department key
max_department_keydf = spark.sql("""
SELECT MAX(department_key) as max_department_key FROM
DQT_Gold_LakeHouse_Layer.department

"""
)

max_department_key = max_department_keydf.collect()[0]['max_department_key']

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 64, Finished, Available)

In [63]:
all_department = new_department.union(changed_department)



# Set start_date
all_department = all_department.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("department_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
all_department = all_department.withColumn("department_key",max_department_key +  row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("customer_id").orderBy("start_date")

# Set end_date
all_department = all_department.withColumn("end_date", to_timestamp(F.lit("9999-12-31")) )


all_department = all_department.withColumn("is_valid", lit(True))

all_department = all_department.select("department_key", "department_id", "department_name" , "start_date" , "end_date" , "is_valid")



StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 65, Finished, Available)

In [64]:
update_is_valid_department = spark.sql("""
SELECT dd.department_key, dd.department_id, dd.department_name, dd.start_date, dd.end_date, dd.is_valid
FROM  department_data dd JOIN department d 
ON dd.department_id = d.department_id
WHERE dd.is_valid = 'true' AND d.department_name != dd.department_name

""")



StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 66, Finished, Available)

In [65]:
department_data = department_data.exceptAll(update_is_valid_department)

update_is_valid_department = update_is_valid_department.withColumn("end_date", current_timestamp())

update_is_valid_department = update_is_valid_department.withColumn("is_valid", lit(False))


update_is_valid_department.createOrReplaceTempView("update_is_valid_department")


StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 67, Finished, Available)

In [66]:


department_data = department_data.union(update_is_valid_department)
department_data = department_data.union(all_department)

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 68, Finished, Available)

# manage category

In [67]:
dim_category = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.category")
dim_category.createOrReplaceTempView("dim_category")


StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 69, Finished, Available)

In [68]:
# register teamview
category.createOrReplaceTempView("category")
new_category = spark.sql("""
SELECT * FROM category 
WHERE category_id NOT IN (
    SELECT category_id FROM dim_category
)
""")

changed_category = spark.sql("""
SELECT d.category_id, d.category_name, d.category_id FROM 
category d JOIN dim_category dd ON
d.category_id = dd.category_id 
WHERE d.category_name != dd.category_name
""")

changed_category.createOrReplaceTempView("changed_category")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 70, Finished, Available)

In [69]:
# find max department key
max_dim_categorydf = spark.sql("""
SELECT MAX(category_key) as max_category_key FROM
DQT_Gold_LakeHouse_Layer.category

"""
)

max_category_key = max_dim_categorydf.collect()[0]['max_category_key']

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 71, Finished, Available)

In [70]:
all_category = new_category.union(changed_category)



# Set start_date
all_category = all_category.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("category_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
all_category = all_category.withColumn("category_key",max_category_key +  row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("customer_id").orderBy("start_date")

# Set end_date
all_category = all_category.withColumn("end_date", to_timestamp(F.lit("9999-12-31")) )


all_category = all_category.withColumn("is_valid", lit(True))



all_category = all_category.select("category_key", "category_id", "category_name" , "department_id" , "start_date", "end_date", "is_valid")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 72, Finished, Available)

In [71]:
update_is_valid_category = spark.sql("""
SELECT dc.category_key, dc.category_id, dc.category_name, dc.department_id, dc.start_date, dc.end_date, dc.is_valid
FROM  dim_category dc JOIN category c 
ON dc.category_id = c.category_id
WHERE dc.is_valid = 'true' AND (c.category_name != dc.category_name  OR c.category_id != dc.category_id)

""")
dim_category = dim_category.exceptAll(update_is_valid_category)

update_is_valid_category = update_is_valid_category.withColumn("end_date", current_timestamp())

update_is_valid_category = update_is_valid_category.withColumn("is_valid", lit(False))


update_is_valid_category.createOrReplaceTempView("update_is_valid_category")


StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 73, Finished, Available)

In [72]:



dim_category = dim_category.union(update_is_valid_category)
dim_category = dim_category.union(all_category)

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 74, Finished, Available)

# load to gold


In [73]:
dim_customer.write.mode("overwrite").format("delta").saveAsTable("dim_customer")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 75, Finished, Available)

In [74]:
dim_product.write.mode("overwrite").format("delta").saveAsTable("dim_product")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 76, Finished, Available)

In [75]:
department_data.write.mode("overwrite").format("delta").saveAsTable("department")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 77, Finished, Available)

In [76]:
dim_category.write.mode("overwrite").format("delta").saveAsTable("category")

StatementMeta(, 2c882422-773c-4fbe-9c9a-516e683e8dd5, 78, Finished, Available)